In [1]:
library(xlsx)
library(readxl)
library(stringr)
library("dplyr")


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
path = getwd()
path = substr(path, 1, nchar(path)-9)
setwd(paste0(path, "data"))

# Read in Data

In [14]:
path = paste0(path, "data/AllPhones1009.xlsx")
samsung = read.xlsx(path, "samsung")
xiaomi = read.xlsx(path, "xiaomi")
apple = read.xlsx(path, "AAPL")
huawei = read.xlsx(path, "KMCACZ CH Equity")
data = rbind(samsung,xiaomi, apple, huawei)

In [15]:
data[0:1,]

com,sup,cus,rel_V_fr_S,rel_V_to_C
005930 KS Equity,018260 KS Equity,AAPL US Equity,1645.8388879091,3563.0885546805


### Generate Node Request for Nodes not in "Nodes_Data.csv"

In [17]:
data = data %>% mutate_at(vars(com, sup, cus), as.character)
all_companies_ticker = unique(c(data$com, data$sup, data$cus))

existing_nodes = read.csv("Data_Dict/Nodes_Data.csv")
existing_nodes = as.character(existing_nodes$ticker)

new_ticker = setdiff(all_companies_ticker, existing_nodes)

access_com_info = data.frame("ticker" = new_ticker)
access_com_info = access_com_info %>% filter(ticker!= 0 &
                                             ticker!= "#N/A N/A",
                                             ticker!= "#N/A Invalid Security",
                                             ticker!= "#N/A Requesting Data...")

get_Name="=IF(ISBLANK(A2),\"\",BDP(A2, \"LONG_COMP_NAME\",\"\"))"
get_country="=IF(ISBLANK(A2),\"\",BDP(A2, \"CNTRY_OF_DOMICILE\",\"\"))"
get_gics="=IF(ISBLANK(A2),\"\",BDP(A2, \"GICS_INDUSTRY_GROUP_NAME\",\"\"))"


access_com_info$Name = NA
access_com_info$country = NA
access_com_info$gics = NA
for(i in 1:(nrow(access_com_info))){
  company = paste0("A", i+1)
  access_com_info$Name[i] = gsub("A2",company, get_Name)
  access_com_info$country[i] = gsub("A2",company, get_country)
  access_com_info$gics[i] = gsub("A2",company, get_gics)
}

# write.csv(access_com_info, "Request/Node_info.csv", row.names = F)
write.xlsx(access_com_info, "Request/Node_info.xlsx", row.names = F)
access_com_info[0:1,]

ticker,Name,country,gics
300288 CH Equity,"=IF(ISBLANK(A2),"""",BDP(A2, ""LONG_COMP_NAME"",""""))","=IF(ISBLANK(A2),"""",BDP(A2, ""CNTRY_OF_DOMICILE"",""""))","=IF(ISBLANK(A2),"""",BDP(A2, ""GICS_INDUSTRY_GROUP_NAME"",""""))"


### With new Node_Info, update "Nodes_Data.csv"

In [ ]:
update = read.xlsx("Request/Node_info.xlsx", 2)
current = read.csv("Data_Dict/Nodes_Data.csv")
data = rbind(current, update)
data=data[!is.na(data$id) & data$id!="" & data$id!="#N/A Invalid Security",]
#Nodes$id <- str_replace_all(Nodes$id," ","_")
data=data[!duplicated(data),]
write.csv(data, "Data_Dict/Nodes_Data.csv", row.names = F)

### Combine All Nodes Data

# Format and Clean Data

#### The following block is for importing data with different formats

In [19]:
#data = rbind(samsung,xiaomi, apple, huawei)
data <- read.xlsx("Huawei.xlsx", 2)
data[0:1,]

com,com_N,sup,sup_N,cus,cus_N,rel_V_fr_S,rel_V_to_C,sup_country,sup_gics,cus_country,cus_gics
KMCACZ CH Equity,Huawei Investment & Holding Co Ltd,601138 CH Equity,Foxconn Industrial Internet Co Ltd,941 HK Equity,China Mobile Ltd,1585.7283218852,1117.21200070292,CN,Technology Hardware & Equipmen,HK,Telecommunication Services


#### For old formats

In [20]:
data = data %>% mutate_all(as.character)

supply_n = data[c(3,4,9,10)]
cus_n = data[c(5,6,11,12)]
colnames(supply_n)=c("ticker","id","country","type_label")
colnames(cus_n)=c("ticker","id","country","type_label")
Nodes = rbind(supply_n, cus_n)
remove(supply_n, cus_n)

supply = data[c(2,4,7)]
cus = data[c(6,2,8)]
colnames(supply)=c("to","from","weight")
colnames(cus)=c("to","from","weight")
Edges = rbind(supply, cus)
remove(cus, supply)

Nodes[0:1,]
Edges[0:1,]

ticker,id,country,type_label
601138 CH Equity,Foxconn Industrial Internet Co Ltd,CN,Technology Hardware & Equipmen


to,from,weight
Huawei Investment & Holding Co Ltd,Foxconn Industrial Internet Co Ltd,1585.7283218852


In [22]:
Edges=Edges[!is.na(Edges$from) & Edges$from!="" & Edges$from!="#N/A Invalid Security",]
Edges=Edges[!is.na(Edges$to) & Edges$to!="" & Edges$to!="#N/A Invalid Security",]
#Edges$from <- str_replace_all(Edges$from," ","_")
#Edges$to <- str_replace_all(Edges$to," ","_")
Edges=Edges[!duplicated(Edges),]

Nodes=Nodes[!is.na(Nodes$id) & Nodes$id!="" & Nodes$id!="#N/A Invalid Security",]
#Nodes$id <- str_replace_all(Nodes$id," ","_")
Nodes=Nodes[!duplicated(Nodes),]

#check complete
for(com in Edges$from){
  if(!(com %in% Nodes$id)){
    print(com)
  }
}
for(com in Edges$to){
  if(!(com %in% Nodes$id)){
    print(com)
  }
}

In [25]:
Nodes_Data = Nodes
Edges_Data = Edges
Nodes_Data[0:1,]
Edges_Data[0:1,]

ticker,id,country,type_label
601138 CH Equity,Foxconn Industrial Internet Co Ltd,CN,Technology Hardware & Equipmen


to,from,weight
Huawei Investment & Holding Co Ltd,Foxconn Industrial Internet Co Ltd,1585.7283218852


In [32]:
path = getwd()
setwd(paste0(path, "/Temp"))
saveRDS(Nodes_Data, file = "Nodes_Data.rds")
saveRDS(Edges_Data, file = "Edges_Data.rds")
